# Steam API 

In [161]:
from steam import WebAPI
import json
import random
import datetime
from pymongo import MongoClient
import time

client = MongoClient()
db = client.steam


In [106]:
#Generating the ID list for sampling

#constant
init = 76561197960265728
end = 76561198422588320

#parameter
bucket = 1000
numOfBuckets = 50
numPerBucket = 1000
width = (end-init)//bucket
print("width is: %d " % width)
now = datetime.datetime.now()

idsFilename = 'idList_' + now.strftime("%Y-%m-%d %H-%M")
bucketsFilename ="bucketsList_" + now.strftime("%Y-%m-%d %H-%M")


def generate_no_replacement(idfilename,bucketfilename):
    bucketFile = open(bucketsFilename,'w')
    with open(idfilename,'w')as idFile:
        for i in random.sample(range(bucket),numOfBuckets):
            bucketFile.write(str(i) + '\n')
            rand = random.sample(range(width),numPerBucket)
            ids = [init+i*width+j for j in rand]
            for n in ids:
                idFile.write(str(n)+'\n')
    idFile.close()
    bucketFile.close()

def generate_replacement(idfilename,bucketfilename):
    bucketFile = open(bucketsFilename,'w')
    with open(idfilename,'w')as idFile:
        for i in range(numOfBuckets):
            k = random.choice(range(bucket))
            bucketFile.write(str(k) + '\n')
            for j in range(numPerBucket):
                n = init+k*width+random.choice(range(width))
                idFile.write(str(n)+'\n')
    idFile.close()
    bucketFile.close()


width is: 462322 


In [107]:
#generate_no_replacement(filename, bucketsFilename)
generate_replacement(idsFilename, bucketsFilename)

In [54]:
key="F586721EBB57C09738BFA05583825173"
api = WebAPI(key=key)

ISteamUser

GetPlayerSummaries

steamids=76561198110726588,76561198422588320

steamids
Comma-delimited list of 64 bit Steam IDs to return profile information for. Up to 100 Steam IDs can be requested.

communityvisibilitystate : 
This represents whether the profile is visible or not, and if it is visible, why you are allowed to see it. Note that because this WebAPI does not use authentication, there are only two possible values returned: 1 - the profile is not visible to you (Private, Friends Only, etc), 3 - the profile is "Public", and the data is visible.


** Get player summary - test **

In [160]:
response = api.ISteamUser.GetPlayerSummaries(steamids="76561198207709726")
print(json.dumps(response, indent=2))

{
  "response": {
    "players": []
  }
}


In [102]:
response = api.ISteamUser.GetPlayerSummaries(steamids="76561198110626464,76561198110626467")
print(json.dumps(response['response']['players'], indent=2))

[
  {
    "steamid": "76561198110626464",
    "communityvisibilitystate": 2,
    "profilestate": 1,
    "personaname": "Jahah",
    "lastlogoff": 1489929966,
    "profileurl": "http://steamcommunity.com/id/jaharay/",
    "avatar": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/d4/d4cabbe77e72e9f7204ae33535e4264d746a50f8.jpg",
    "avatarmedium": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/d4/d4cabbe77e72e9f7204ae33535e4264d746a50f8_medium.jpg",
    "avatarfull": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/d4/d4cabbe77e72e9f7204ae33535e4264d746a50f8_full.jpg",
    "personastate": 0
  },
  {
    "steamid": "76561198110626467",
    "communityvisibilitystate": 3,
    "personaname": "maks-ivanov-2002",
    "profileurl": "http://steamcommunity.com/profiles/76561198110626467/",
    "avatar": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb.jpg",
    "avat

In [103]:
result = db.GetPlayerSummaries.insert_many(response['response']['players'])

76561198110626464 

communityvisibilitystate": 2 ( 3 is public, 1 is private)

list of games is empty! 

we still able to see name and lastlogoff.

** get list of games ** 

In [195]:
response = api.IPlayerService.GetOwnedGames(steamid=76561198207709727, include_appinfo=1, include_played_free_games=0, appids_filter=[])
response['2'] = 'kk'
response

{'2': 'kk',
 'response': {'game_count': 10,
  'games': [{'appid': 10,
    'has_community_visible_stats': True,
    'img_icon_url': '6b0312cda02f5f777efa2f3318c307ff9acafbb5',
    'img_logo_url': 'af890f848dd606ac2fd4415de3c3f5e7a66fcb9f',
    'name': 'Counter-Strike',
    'playtime_forever': 573},
   {'appid': 80,
    'img_icon_url': '077b050ef3e89cd84e2c5a575d78d53b54058236',
    'img_logo_url': 'acdb28ba1b4c2fcc526332c1b63fc0f7533f087f',
    'name': 'Counter-Strike: Condition Zero',
    'playtime_forever': 0},
   {'appid': 100,
    'img_icon_url': '077b050ef3e89cd84e2c5a575d78d53b54058236',
    'img_logo_url': 'acdb28ba1b4c2fcc526332c1b63fc0f7533f087f',
    'name': 'Counter-Strike: Condition Zero Deleted Scenes',
    'playtime_forever': 0},
   {'appid': 253710,
    'has_community_visible_stats': True,
    'img_icon_url': '4a91dc770675e559038a4f46e0daa4f8fa42ee30',
    'img_logo_url': 'fbfd9daf55e8489e42500bd3e08ca8f4c5f028db',
    'name': 'theHunter Classic',
    'playtime_forever': 

In [202]:
# get all games but include free gmaes (not games 76561198110526580)
response = api.IPlayerService.GetOwnedGames(steamid=76561198110526572, include_appinfo=1, include_played_free_games=1, appids_filter=[])
print(json.dumps(response, indent=2))

#notice playtime is in mintues not hours

{
  "response": {
    "game_count": 2,
    "games": [
      {
        "appid": 205790,
        "name": "Dota 2 Test",
        "playtime_forever": 0,
        "img_icon_url": "",
        "img_logo_url": ""
      },
      {
        "appid": 570,
        "name": "Dota 2",
        "playtime_forever": 2703,
        "img_icon_url": "0bbb630d63262dd66d2fdd0f7d37e8661a410075",
        "img_logo_url": "d4f836839254be08d8e9dd333ecc9a01782c26d2"
      }
    ]
  }
}


In [170]:
# go over all collected list, but the ids in chuncks of 100. and collect all players details. 
f = open('idList_2017-09-23 03-08', 'r')
x = f.readlines()
print (len(x))
stripX = [i.strip('\n') for i in x]

chunkSize = 100
l = [ stripX[i:i + chunkSize] for i in range(0, len(stripX), chunkSize) ]

queriedIdCounter = 0
responseCounter = 0
for i in l:
    steamChunckIds = ','.join(map(str, i))
    response = api.ISteamUser.GetPlayerSummaries(steamids=steamChunckIds)
    db.GetPlayerSummaries.insert_many(response['response']['players'])
    queriedIdCounter += len(i)
    responseCounter += len(response['response']['players'])
    time.sleep(.250)

print("#queried Ids: {} - #responsed: {}".format(queriedIdCounter, responseCounter))

50000
#queried Ids: 50000 - #responsed: 45709


In [ ]:
queriedIdCounter1 = 0
responseCounter1 = 0

for i in stripX:
    response = api.IPlayerService.GetOwnedGames(steamid=i, include_appinfo=1, include_played_free_games=0, appids_filter=[])
    queriedIdCounter1 +=1
    if response['response'] != {}:
        responseCounter1 += 1
        response['playerId'] = i
        db.GetOwnedGames.insert_one(response)

print("#queried Ids: {} - #responsed: {}".format(queriedIdCounter1, responseCounter1))        

    

32587


# get app price ( does'nt need authentication) 

You can get all apps (games), and the IDs for it, by using this API:

http://api.steampowered.com/ISteamApps/GetAppList/v0001/
You will get nearly every information you need about a game (including screenshots).

Region

You can change the region by adding the parameter cc.

For example:

http://store.steampowered.com/api/appdetails?appids=57690&cc=us
http://store.steampowered.com/api/appdetails?appids=57690&cc=gb
http://store.steampowered.com/api/appdetails?appids=57690&cc=de

Appid 57690 = Tropico 4

http://store.steampowered.com/api/appdetails?appids=57690
Part of the response:

"price_overview":{
    "currency":"EUR",
    "initial":2999,
    "final":2999,
    "discount_percent":0
}


2999 is likely 29.99

